<a href="https://colab.research.google.com/github/pnalaba/DeployDemo/blob/master/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyfarmhash

     |████████████████████████████████| 102kB 4.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/3f/41/10/80e03a3ffa9ba29a41eac65c4371ade0bd1e21dbdabb867c31
Successfully built pyfarmhash


In [3]:
!pip install tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 1.3MB/s 
     |████████████████████████████████| 501kB 43.3MB/s 
     |████████████████████████████████| 3.1MB 38.0MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [4]:
import numpy as np
import pandas as pd
import farmhash
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.externals import joblib
from scipy import stats
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.client import device_lib
import os
from datetime import datetime
import time

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [19]:
df_test=pd.read_csv("df_test.csv")

ParserError: ignored

In [0]:
ROOT_DIR="/content/drive/My Drive/Colab Notebooks/ds_project/"


In [0]:
CATEGORICAL_COLS_FILE = os.path.join(ROOT_DIR,"categorical_cols.txt")
IMP_MEAN_SAVE = os.path.join(ROOT_DIR,"imp_mean.save")
SCALER_SAVE = os.path.join(ROOT_DIR,"scaler.save")
PCA_SAVE = os.path.join(ROOT_DIR,"pca.save")

In [0]:
fp=open(CATEGORICAL_COLS_FILE)
categorical_cols = fp.read().split(",")

In [15]:
df_y_test = df_test["y"]
df_test =df_test.drop(columns=categorical_cols+["y"])
cols_before_norm = df_test.columns
imp_mean = joblib.load(IMP_MEAN_SAVE)
df_interpolated = imp_mean.transform(df_test)
sc = joblib.load(SCALER_SAVE)
na_norm = sc.transform(df_interpolated)


ValueError: ignored

In [18]:
df_test.columns

Index(['Unnamed: 0', 'x001', 'x002', 'x003', 'x004', 'x005', 'x006', 'x007',
       'x008', 'x009',
       ...
       'x299', 'x300', 'x301', 'x302', 'x303', 'x304', 'y', 'train', 'eval',
       'test'],
      dtype='object', length=309)

In [0]:
fc = tf.feature_column
  
def one_hot_cat_column(feature_name, vocab):
    return fc.indicator_column(
      fc.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))


In [0]:
def make_input_fn(X):
    def input_fn():
        dataset = tf.data.Dataset.from_tensor(X)
        return dataset
    return input_fn

In [0]:
pca_99 = joblib.load(os.path.join(ROOT_DIR,"pca_99.save"))
na_pca_99 = pca_99.transform(na_norm)
df_pca_99 = pd.DataFrame(na_pca_99, columns=("px{0:03}".format(i) for i in range(na_pca_99.shape[1])) )

In [0]:
feature_columns_99 = []
  
for feature_name in df_pca_99.columns:  
    feature_columns_99.append(fc.numeric_column(feature_name,
                                          dtype=tf.float32))

In [0]:
# Training and evaluation input functions.
input_fn_99 = make_input_fn(df_pca_99)


model_dir = os.path.join(ROOT_DIR,"boosted_trees_pca99_unscaled/")

with tf.device('/device:GPU:0') :
    est = tf.estimator.BoostedTreesRegressor(feature_columns_99,
                                        model_dir=model_dir,
                                        n_batches_per_layer=n_batches,                                   
                                             l2_regularization=0.01,
                                             n_trees=150,
                                              quantile_sketch_epsilon=1.0/512,
                                             max_depth=9
                                          )

In [0]:
predictions_99 = [round(p['predictions'][0]) for p in est_99.predict(input_fn_99) ]
actual_99 = df_y_test

In [0]:
df_pred_99 = pd.DataFrame({'p' : predictions_99, 'a' : actual_99})
df_pred_99["abs_e"] = np.abs(df_pred_99["p"] - df_pred_99["a"])
df_pred_99["square_e"] = np.square(df_pred_99["abs_e"]) 

In [0]:
rmse = np.sqrt(1774.521)
accuracy = len(df_pred_99[df_pred_99["abs_e"]<=3])/len(df_pred_99)
print(rmse,accuracy)